In [1]:
import gymnasium as gym
from wrappers import TltlWrapper, PredicateEvaluationResult, PositionPredicate
from gymnasium.spaces.utils import flatten_space, flatten
from gymnasium.spaces import Dict, Discrete
from minigrid.wrappers import DictObservationSpaceWrapper, FullyObsWrapper
from stable_baselines3.common.logger import configure
import numpy as np

pygame 2.5.1 (SDL 2.28.2, Python 3.9.17)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
env = gym.make("MiniGrid-Empty-8x8-v0", render_mode="rgb_array", max_episode_steps=20)
env = FullyObsWrapper(env)
env = TltlWrapper(env, tltl="F p1 & F p2 & (! p2 U p1)", predicates={'p1': PositionPredicate(True, [5, 5]), 'p2': PositionPredicate(True, [3, 3])})
print(env.fspa)
env = gym.wrappers.FilterObservation(env, ["image", "fspa", "direction"])
env = gym.experimental.wrappers.LambdaObservationV0(env, lambda obs: {**obs, 'image': flatten(env.observation_space['image'],obs['image'])}, gym.spaces.Dict({**env.observation_space.spaces, "image": flatten_space(env.observation_space['image'])}))
print(env.action_space, env.observation_space)


Name: Fspa corresponding to the formula: F p1 & F p2 & (! p2 U p1)
Directed: True
Multi: True
Props: {'p1': 1, 'p2': 2}
Alphabet: {0, 1, 2, 3}
Initial: ['T0_init']
Final: {'accept_all'}
Trap: {'T0_S3'}
Nodes: [('T0_init', {}), ('accept_all', {}), ('T0_S2', {}), ('T0_S3', {})]
Edges: [('T0_init', 'accept_all', {'weight': 0, 'input': {3}, 'guard': '((p1) && (p2))', 'label': '((p1) && (p2))'}), ('T0_init', 'T0_init', {'weight': 0, 'input': {0}, 'guard': '((!(p1)) && (!(p2)))', 'label': '((!(p1)) && (!(p2)))'}), ('T0_init', 'T0_S2', {'weight': 0, 'input': {1}, 'guard': '((p1) && (!(p2)))', 'label': '((p1) && (!(p2)))'}), ('T0_init', 'T0_S3', {'weight': 0, 'input': {2}, 'guard': '((!(p1)) && (p2))', 'label': '((!(p1)) && (p2))'}), ('accept_all', 'accept_all', {'weight': 0, 'input': {0, 1, 2, 3}, 'guard': '(1)', 'label': '(1)'}), ('T0_S2', 'accept_all', {'weight': 0, 'input': {2, 3}, 'guard': '(p2)', 'label': '(p2)'}), ('T0_S2', 'T0_S2', {'weight': 0, 'input': {0, 1}, 'guard': '(!(p2))', 'l

In [3]:
#print(np.array(env.observation_space['image'].shape) ,np.array(env.observation_space['fspa'].size))

In [4]:
from stable_baselines3 import A2C, PPO
from stable_baselines3.common.atari_wrappers import ClipRewardEnv

In [5]:
model = PPO("MultiInputPolicy", env, verbose=1)
tmp_path = "./tmp/sb3_log/"
new_logger = configure(tmp_path, ["stdout", "csv", "tensorboard"])
model.set_logger(new_logger)
model.learn(total_timesteps=300000)
model.save('a2c4')

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./tmp/sb3_log/
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 19.9     |
|    ep_rew_mean     | -102     |
| time/              |          |
|    fps             | 527      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 19.7        |
|    ep_rew_mean          | -110        |
| time/                   |             |
|    fps                  | 485         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009723673 |
|    clip_fraction        | 0.0562      |
|    clip_range           | 0.2    

In [ ]:

# #obs, info =env.reset()
vec_env = model.get_env()
obs = vec_env.reset()
for i in range(0, 1000):
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, info = vec_env.step(action)
    print(action,reward, obs['fspa'], done, vec_env.get_attr('agent_pos'))
    # VecEnv resets automatically
    if done:
      obs = vec_env.reset()


In [ ]:
# print(env.fspa_all_states)
# obs, info =env.reset()
# for i in range(1000):
#     action = env.action_space.sample()
#     obs, reward, done, truncated, info = env.step(action)
#     print('action', action, "fspa", obs['fspa'],'position', env.agent_pos ,'reward', reward)